In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment

def scrape_fantasy_season(year):
    url = f'https://www.pro-football-reference.com/years/{year}/fantasy.htm'
    print(f"🔄 Scraping {year}...")
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Try direct table
    table = soup.find('table', id='fantasy')

    # Try inside comments if not found
    if table is None:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            if 'id="fantasy"' in comment:
                table = BeautifulSoup(comment, 'html.parser').find('table', id='fantasy')
                break

    if table is None:
        print(f"❌ {year} failed: No fantasy table found.")
        return None

    df = pd.read_html(str(table))[0]
    df['Season'] = year
    return df

# Loop through 2017–2024
all_years = []
for yr in range(2017, 2025):
    try:
        df = scrape_fantasy_season(yr)
        if df is not None:
            all_years.append(df)
    except Exception as e:
        print(f"⚠️ Error in {yr}: {e}")

# Combine all years
if all_years:
    fantasy_df = pd.concat(all_years, ignore_index=True)
    print(f"✅ Combined DataFrame shape: {fantasy_df.shape}")
    # fantasy_df.to_csv("fantasy_2017_2024.csv", index=False)
    display(fantasy_df.head())
else:
    print("❌ No data was scraped.")


🔄 Scraping 2017...


C:\Users\Carl\AppData\Local\Temp\ipykernel_9688\1311803391.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


🔄 Scraping 2018...


C:\Users\Carl\AppData\Local\Temp\ipykernel_9688\1311803391.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


🔄 Scraping 2019...


...

C:\Users\Carl\AppData\Local\Temp\ipykernel_9688\1311803391.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


🔄 Scraping 2024...


C:\Users\Carl\AppData\Local\Temp\ipykernel_9688\1311803391.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


✅ Combined DataFrame shape: (5264, 34)


Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player                 Tm            FantPos   
0                  1      Todd Gurley*+                LAR                 RB   
1                  2     Le'Veon Bell*+                PIT                 RB   
2                  3       Kareem Hunt*                KAN                 RB   
3                  4      Alvin Kamara*                NOR                 RB   
4                  5      Melvin Gordon                LAC                 RB   

  Unnamed: 4_level_0 Games     Passing          ... Scoring      Fantasy  \
                 Age     G  GS     Cmp Att Yds  ...     2PM  2PP  FantPt   
0                 23    15  15       0   0   0  ...     NaN  NaN     319   
1                 25    15  15       0   0   0  ...     NaN  NaN     257   
2                 22    16  16       0   0   0  ...     NaN  NaN     242   
3                 22    16   3       0   0   0  ...       1  NaN     239   
4                 24    16  16       0   0   0  ...     NaN  NaN     230   

                                           Season  
     PPR   DKPt   FDPt  VBD PosRank OvRank         
0  383.3  391.3  351.3  192       1      1   2017  
1  341.6  349.6  299.1  130       2      2   2017  
2  295.2  302.2  268.7  115       3      3   2017  
3  320.4  327.4  279.9  112       4      4   2017  
4  288.1  294.1  259.1  103       5      5   2017  

[5 rows x 34 columns]

In [2]:
def normalize_column_names(df):
    df.columns = (
        df.columns
        .str.lower()
        .str.replace(' ', '_')
        .str.replace(r'[^\w_]', '', regex=True)
    )

    rename_map = {
        'player': 'player',
        'tm': 'team',
        'fantasy_fantpt': 'fantasy_points',
        'fantasy_posrank': 'pos_rank',
        'fantasy_ovrank': 'ovr_rank',
        'rushing_yds': 'rush_yds',
        'rushing_td': 'rush_td',
        'receiving_yds': 'rec_yds',
        'receiving_td': 'rec_td',
        'passing_yds': 'pass_yds',
        'passing_td': 'pass_td',
        'passing_int': 'interceptions',
        'season': 'season'
    }

    df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)
    return df


In [3]:
# If any tables had multi-level column headers, flatten them
# Flatten MultiIndex columns into single strings
fantasy_df.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in fantasy_df.columns]

# Check the first 10 column names to confirm
print("✅ Flattened Columns:\n", fantasy_df.columns[:10])

✅ Flattened Columns:
 Index(['Unnamed: 0_level_0 Rk', 'Unnamed: 1_level_0 Player',
       'Unnamed: 2_level_0 Tm', 'Unnamed: 3_level_0 FantPos',
       'Unnamed: 4_level_0 Age', 'Games G', 'Games GS', 'Passing Cmp',
       'Passing Att', 'Passing Yds'],
      dtype='object')


In [22]:
fantasy_df = fantasy_df.rename(columns={
    'Fantasy FantPt': 'FantPt',
    'Fumbles Fmb': 'Fumbles',
    'Games G': 'Games',
    'Unnamed: 0_level_0 Rk': 'Rk',
    'Unnamed: 1_level_0 Player': 'Player',
    'Unnamed: 2_level_0 Tm': 'Team',
    'Unnamed: 3_level_0 FantPos': 'Position',
    'Unnamed: 4_level_0 Age': 'Age'
})

In [23]:
print(fantasy_df.columns.tolist())

['Rk', 'Player', 'Team', 'Position', 'Age', 'Games', 'Games GS', 'Passing Cmp', 'Passing Att', 'Passing Yds', 'Passing TD', 'Passing Int', 'Rushing Att', 'Rushing Yds', 'Rushing Y/A', 'Rushing TD', 'Receiving Tgt', 'Receiving Rec', 'Receiving Yds', 'Receiving Y/R', 'Receiving TD', 'Fumbles', 'Fumbles FL', 'Scoring TD', 'Scoring 2PM', 'Scoring 2PP', 'FantPt', 'Fantasy PPR', 'Fantasy DKPt', 'Fantasy FDPt', 'Fantasy VBD', 'Fantasy PosRank', 'Fantasy OvRank', 'Season']


In [24]:
print(fantasy_df[['Player', 'FantPt', 'Fumbles', 'Position', 'Age']].head())

           Player  FantPt  Fumbles Position Age
0   Todd Gurley*+     319        5       RB  23
1  Le'Veon Bell*+     257        3       RB  25
2    Kareem Hunt*     242        1       RB  22
3   Alvin Kamara*     239        1       RB  22
4   Melvin Gordon     230        1       RB  24


In [25]:
# Drop rows with missing or repeated headers
fantasy_df = fantasy_df[fantasy_df['Player'].notna()]
fantasy_df = fantasy_df[fantasy_df['Player'] != 'Player']

# Fill NaNs
fantasy_df['FantPt'] = fantasy_df['FantPt'].fillna(0)
fantasy_df['Fumbles'] = fantasy_df['Fumbles'].fillna(0)

# Cast numeric columns
fantasy_df['FantPt'] = pd.to_numeric(fantasy_df['FantPt'], errors='coerce')
fantasy_df['Fumbles'] = pd.to_numeric(fantasy_df['Fumbles'], errors='coerce')

In [37]:
print(fantasy_df[['Player']].value_counts().head())


Player        
Chris Moore       9
Nelson Agholor    8
Allen Robinson    8
Johnny Mundt      8
Gerald Everett    8
Name: count, dtype: int64


In [28]:
season_dfs = {
    season: fantasy_df[fantasy_df['Season'] == season].reset_index(drop=True)
    for season in fantasy_df['Season'].unique()
}

In [35]:
season_dfs[2018]['Player'].value_counts().head()

Player
Todd Gurley*+    1
Kenjon Barner    1
Corey Coleman    1
Tyler Conklin    1
Matt Lengel      1
Name: count, dtype: int64